In [1]:
import numpy as np
import pandas as pd
from pandas_streaming.df import StreamingDataFrame
import os

from transformers import AutoTokenizer, TrainingArguments, Trainer, ModernBertForSequenceClassification
from datasets import Dataset, load_dataset


import evaluate
import torch

torch.set_float32_matmul_precision('high')

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # convert the logits to their predicted class
    predictions = np.argmax(logits, axis=-1)
    
    # print(f"Avg pred: {sum(predictions)/len(predictions)}, Avg Labels: {sum(labels)/len(labels)}")
    
    return metric.compute(predictions=predictions, references=labels)

dataset = load_dataset("csv", data_files="../data/pairwise-model-data/pairedwise-tweets.tsv", delimiter="\t")
dataset.shuffle()
split = dataset["train"].train_test_split(test_size=0.2)

train_dataset = split["train"].remove_columns('Unnamed: 0').take(32000)
eval_dataset = split["test"].remove_columns('Unnamed: 0').take(8000)



In [2]:
def preprocess_function(examples):
    full_str = "[CLS] "
    full_str += examples["query"]
    
    full_str += " [SEP] " + examples["paper1"]
    full_str += " [SEP] " + examples["paper2"]

    return tokenizer(full_str, truncation=True).to('cuda')


id2label = {0: "0", 1: "1"}
label2id = {"0": 0, "1": 1}

tokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-base")

In [3]:
train_dataset_tokenized = train_dataset.map(preprocess_function)

Map:   0%|          | 0/32000 [00:00<?, ? examples/s]

In [4]:
test_dataset_tokenized = eval_dataset.map(preprocess_function)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [5]:
model = ModernBertForSequenceClassification.from_pretrained("answerdotai/ModernBERT-base", num_labels=2, id2label=id2label, label2id=label2id).to("cuda")

training_args = TrainingArguments(
    output_dir="../models/pairwise-classifier-full",
    learning_rate=2e-6,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="steps",
    eval_steps=4000,
    save_strategy="steps",
    save_steps=4000,
    load_best_model_at_end=True,
    push_to_hub=False,
    fp16=True,
    optim="adamw_torch",
    save_total_limit=2,
    eval_on_start=False,
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_tokenized,
    eval_dataset=test_dataset_tokenized,
    compute_metrics=compute_metrics,
)

# trainer.evaluate()
trainer.train(resume_from_checkpoint=True)

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
W0527 23:24:38.681000 1751 torch/_inductor/utils.py:1250] [1/0] Not enough SMs to use max_autotune_gemm mode


Step,Training Loss,Validation Loss,Accuracy
48000,0.535500,0.567450,0.897250
52000,0.468800,0.466353,0.918375
56000,0.544200,0.457961,0.921625
60000,0.373300,0.474057,0.921625
64000,0.392500,0.463291,0.922500


TrainOutput(global_step=64000, training_loss=0.15793819427490235, metrics={'train_runtime': 36830.5612, 'train_samples_per_second': 1.738, 'train_steps_per_second': 1.738, 'total_flos': 4.11663284359997e+16, 'train_loss': 0.15793819427490235, 'epoch': 2.0})